<a href="https://colab.research.google.com/github/dinky-coder/dinky-coder/blob/main/The_Capability_Architectural_Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Capability Architectural Pattern
# Extending the Agent Loop with Capabilities
# The Capability Pattern

In [10]:
from functools import reduce

def run(self, user_input: str, memory=None, action_context_props=None):

    # Initialize capabilities
    for capability in self.capabilities:
        capability.init(self, action_context)

    while True:
        # Start of loop capabilities
        can_start_loop = reduce(lambda a, c: c.start_agent_loop(self, action_context),
                              self.capabilities, False)

        # Construct prompt with capability modifications
        prompt = reduce(lambda p, c: c.process_prompt(self, action_context, p),
                      self.capabilities, base_prompt)

        # Process response with capabilities
        response = reduce(lambda r, c: c.process_response(self, action_context, r),
                    self.capabilities, response)

        # Process action with capabilities
        action = reduce(lambda a, c: c.process_action(self, action_context, a),
                      self.capabilities, action)

        # Process result with capabilities
        result = reduce(lambda r, c: c.process_result(self, action_context, response,
                                                     action_def, action, r),
                       self.capabilities, result)

        # End of loop capabilities
        for capability in self.capabilities:
            capability.end_agent_loop(self, action_context)

Understanding the Capability Class

In [11]:
from typing import List

# Placeholder for ActionContext - replace with actual definition or import
class ActionContext:
    pass

# Placeholder for Prompt - replace with actual definition or import
class Prompt:
    pass

# Placeholder for Action - replace with actual definition or import
class Action:
    pass

# Placeholder for Memory - replace with actual definition or import
class Memory:
    pass

class Capability:
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    def init(self, agent, action_context: ActionContext) -> dict:
        """Called once when the agent starts running."""
        pass

    def start_agent_loop(self, agent, action_context: ActionContext) -> bool:
        """Called at the start of each iteration through the agent loop."""
        return True

    def process_prompt(self, agent, action_context: ActionContext,
                      prompt: Prompt) -> Prompt:
        """Called right before the prompt is sent to the LLM."""
        return prompt

    def process_response(self, agent, action_context: ActionContext,
                        response: str) -> str:
        """Called after getting a response from the LLM."""
        return response

    def process_action(self, agent, action_context: ActionContext,
                      action: dict) -> dict:
        """Called after parsing the response into an action."""
        return action

    def process_result(self, agent, action_context: ActionContext,
                      response: str, action_def: Action,
                      action: dict, result: any) -> any:
        """Called after executing the action."""
        return result

    def process_new_memories(self, agent, action_context: ActionContext,
                           memory: Memory, response, result,
                           memories: List[dict]) -> List[dict]:
        """Called when new memories are being added."""
        return memories

    def end_agent_loop(self, agent, action_context: ActionContext):
        """Called at the end of each iteration through the agent loop."""
        pass

    def should_terminate(self, agent, action_context: ActionContext,
                        response: str) -> bool:
        """Called to check if the agent should stop running."""
        return False

    def terminate(self, agent, action_context: ActionContext) -> dict:
        """Called when the agent is shutting down."""
        pass

Implementing Time Awareness

In [12]:
from datetime import datetime
from zoneinfo import ZoneInfo

class TimeAwareCapability(Capability):
    def __init__(self):
        super().__init__(
            name="Time Awareness",
            description="Allows the agent to be aware of time"
        )

    def init(self, agent, action_context: ActionContext) -> dict:
        """Set up time awareness at the start of agent execution."""
        # Get timezone from context or use default
        time_zone_name = action_context.get("time_zone", "America/Chicago")
        timezone = ZoneInfo(time_zone_name)

        # Get current time in specified timezone
        current_time = datetime.now(timezone)

        # Format time in both machine and human-readable formats
        iso_time = current_time.strftime("%Y-%m-%dT%H:%M:%S%z")
        human_time = current_time.strftime("%H:%M %A, %B %d, %Y")

        # Store time information in memory
        memory = action_context.get_memory()
        memory.add_memory({
            "type": "system",
            "content": f"""Right now, it is {human_time} (ISO: {iso_time}).
            You are in the {time_zone_name} timezone.
            Please consider the day/time, if relevant, when responding."""
        })

    def process_prompt(self, agent, action_context: ActionContext,
                      prompt: Prompt) -> Prompt:
        """Update time information in each prompt."""
        time_zone_name = action_context.get("time_zone", "America/Chicago")
        current_time = datetime.now(ZoneInfo(time_zone_name))

        # Add current time to system message
        system_msg = (f"Current time: "
                     f"{current_time.strftime('%H:%M %A, %B %d, %Y')} "
                     f"({time_zone_name})\n\n")

        # Add to existing system message or create new one
        messages = prompt.messages
        if messages and messages[0]["role"] == "system":
            messages[0]["content"] = system_msg + messages[0]["content"]
        else:
            messages.insert(0, {
                "role": "system",
                "content": system_msg
            })

        return Prompt(messages=messages)

In [13]:
from functools import reduce
from typing import List

# Placeholder definitions for Agent and its dependencies
class Goal:
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

class JSONAgentLanguage:
    pass

class ActionRegistry:
    pass

class PythonEnvironment:
    pass

# Placeholder for llm and registry instances
llm = None # Replace with your actual LLM instance
registry = ActionRegistry() # Replace with your actual registry instance

class Agent:
    def __init__(self, goals: List[Goal], agent_language, action_registry, generate_response, environment, capabilities: List['Capability'] = None):
        self.goals = goals
        self.agent_language = agent_language
        self.action_registry = action_registry
        self.generate_response = generate_response
        self.environment = environment
        self.capabilities = capabilities if capabilities is not None else []
        # Placeholder for action_context and base_prompt, replace with actual initialization
        self.action_context = ActionContext()
        self.base_prompt = Prompt(messages=[{"role": "user", "content": "Initial prompt"}])
        self.response = "" # Placeholder for initial response
        self.action_def = Action() # Placeholder for action definition
        self.action = {} # Placeholder for action
        self.result = None # Placeholder for result


    def run(self, user_input: str, memory=None, action_context_props=None):
        # Assuming action_context is initialized elsewhere or within init
        action_context = self.action_context # Use the initialized action_context

        # Initialize capabilities
        for capability in self.capabilities:
            capability.init(self, action_context)

        while True:
            # Start of loop capabilities
            can_start_loop = reduce(lambda a, c: c.start_agent_loop(self, action_context),
                                  self.capabilities, False)

            # Construct prompt with capability modifications
            prompt = reduce(lambda p, c: c.process_prompt(self, action_context, p),
                          self.capabilities, self.base_prompt) # Use self.base_prompt

            # Assume prompt is sent to LLM and response is received here
            # For demonstration, we'll use a placeholder response
            response = "Placeholder response from LLM"

            # Process response with capabilities
            response = reduce(lambda r, c: c.process_response(self, action_context, r),
                        self.capabilities, response)

            # Assume response is parsed into an action here
            # For demonstration, we'll use a placeholder action
            action = {"name": "placeholder_action", "parameters": {}}

            # Process action with capabilities
            action = reduce(lambda a, c: c.process_action(self, action_context, a),
                          self.capabilities, action)

            # Assume action is executed and result is obtained here
            # For demonstration, we'll use a placeholder result
            result = "Placeholder result from action execution"

            # Process result with capabilities
            result = reduce(lambda r, c: c.process_result(self, action_context, response,
                                                         self.action_def, action, result), # Use self.action_def
                           self.capabilities, result)

            # End of loop capabilities
            for capability in self.capabilities:
                capability.end_agent_loop(self, action_context)

            # Add termination condition to avoid infinite loop for placeholder
            break


# Placeholder for ActionContext, Prompt, Action, Memory - replace with actual definition or import
class ActionContext:
    def get(self, key, default=None):
        # Placeholder for getting values from context
        return default

    def get_memory(self):
        # Placeholder for getting memory
        return Memory()

class Prompt:
    def __init__(self, messages):
        self.messages = messages

class Action:
    pass

class Memory:
    def add_memory(self, memory_item):
        # Placeholder for adding memory
        print(f"Adding memory: {memory_item}")


agent = Agent(
    goals=[Goal(name="task", description="Complete the assigned task")],
    agent_language=JSONAgentLanguage(),
    action_registry=registry,
    generate_response=llm, # Using llm placeholder
    environment=PythonEnvironment(),
    capabilities=[
        TimeAwareCapability()
    ]
)

# Example of running the agent (will use placeholder logic)
# agent.run("Example user input")

Extending the Time Awareness Capability

In [14]:
class EnhancedTimeAwareCapability(TimeAwareCapability):
    def process_action(self, agent, action_context: ActionContext,
                      action: dict) -> dict:
        """Add timing information to action results."""
        # Add execution time to action metadata
        action["execution_time"] = datetime.now(
            ZoneInfo(action_context.get("time_zone", "America/Chicago"))
        ).isoformat()
        return action

    def process_result(self, agent, action_context: ActionContext,
                      response: str, action_def: Action,
                      action: dict, result: any) -> any:
        """Add duration information to results."""
        if isinstance(result, dict):
            result["action_duration"] = (
                datetime.now(ZoneInfo(action_context.get("time_zone"))) -
                datetime.fromisoformat(action["execution_time"])
            ).total_seconds()
        return result